<a href="https://colab.research.google.com/github/khushboolohat/Group-2-final/blob/Akshita/khushboo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#new file

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
! pip install alpha-vantage
from alpha_vantage.timeseries import TimeSeries

     |████████████████████████████████| 1.3MB 6.6MB/s 
     |████████████████████████████████| 296kB 22.8MB/s 
     |████████████████████████████████| 143kB 21.0MB/s 


In [3]:
demo = 'GF6ZHIZTJZE9APJY.' #YOUR API KEY HERE

In [4]:
url_global_deaths = r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
url_global_confirmed = r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_retail_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NYSE:SPG&apikey=' + demo + '&datatype=csv'
url_usa_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=' + demo + '&datatype=csv'
url_canada_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=TSE:BB&apikey=' + demo + '&datatype=csv'
url_travel_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=EXPE&apikey=' + demo + '&datatype=csv'
url_material_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NYSE:GOLD&apikey=' + demo + '&datatype=csv'

In [5]:
deaths_df = pd.read_csv(url_global_deaths)
confirm_df = pd.read_csv(url_global_confirmed)
us_stock = pd.read_csv(url_usa_stock)
ca_stock = pd.read_csv(url_canada_stock)
retail_stock = pd.read_csv(url_retail_stock)
travel_stock = pd.read_csv(url_travel_stock)
material_stock = pd.read_csv(url_material_stock)

In [6]:
print(deaths_df.shape)
print(confirm_df.shape)
print(us_stock.shape)
print(ca_stock.shape)
print(retail_stock.shape)
print(travel_stock.shape)
print(material_stock.shape)

(274, 413)
(274, 413)
(100, 6)
(100, 6)
(100, 6)
(100, 6)
(100, 6)


In [7]:
deaths_df_long = pd.melt(deaths_df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                        value_vars= deaths_df.columns[4:], var_name='timestamp', value_name='death_cases')

confirm_df_long = pd.melt(confirm_df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                        value_vars= confirm_df.columns[4:], var_name='timestamp', value_name='confirmed_cases')

In [9]:
deaths_df_long.head()

,Province/State,Country/Region,Lat,Long,timestamp,death_cases
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [8]:
confirm_df_long.head()

,Province/State,Country/Region,Lat,Long,timestamp,confirmed_cases
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [10]:
us_stock['stock_name'] = 'MSFT'
ca_stock['stock_name'] = 'BB'
retail_stock['stock_name'] = 'SPG'
travel_stock['stock_name'] = 'EXPE'
material_stock['stock_name'] = 'GOLD'

In [11]:
final_stock = pd.concat([us_stock, ca_stock, retail_stock, travel_stock, material_stock]).reset_index(drop = True)
final_stock.stock_name.unique()

array(['MSFT', 'BB', 'SPG', 'EXPE', 'GOLD'], dtype=object)

In [12]:
final_stock.shape

(500, 7)

In [13]:
final_stock.tail()

,timestamp,open,high,low,close,volume,stock_name
495,2020-10-16,27.86,27.89,27.50,27.57,8991481,GOLD
496,2020-10-15,27.97,28.08,27.56,27.75,10768733,GOLD
497,2020-10-14,28.20,28.59,27.95,28.31,10603437,GOLD
498,2020-10-13,27.80,28.06,27.18,27.96,12480095,GOLD
499,2020-10-12,27.95,28.19,27.80,28.04,9850833,GOLD


In [14]:
deaths_df_long = deaths_df_long.groupby(['Country/Region','timestamp'])['death_cases'].sum().reset_index()
confirm_df_long = confirm_df_long.groupby(['Country/Region','timestamp'])['confirmed_cases'].sum().reset_index()
final_covid = pd.concat([deaths_df_long, confirm_df_long['confirmed_cases']], axis = 1)

In [15]:
final_covid.head()

,Country/Region,timestamp,death_cases,confirmed_cases
0,Afghanistan,1/1/21,2191,51526
1,Afghanistan,1/10/21,2277,53489
2,Afghanistan,1/11/21,2288,53538
3,Afghanistan,1/12/21,2301,53584
4,Afghanistan,1/13/21,2301,53584


In [16]:
final_covid.tail()

,Country/Region,timestamp,death_cases,confirmed_cases
78523,Zimbabwe,9/5/20,206,6837
78524,Zimbabwe,9/6/20,206,6837
78525,Zimbabwe,9/7/20,210,7298
78526,Zimbabwe,9/8/20,218,7388
78527,Zimbabwe,9/9/20,222,7429


In [17]:
final_covid['timestamp'] = pd.to_datetime(final_covid.timestamp)
final_stock['timestamp'] = pd.to_datetime(final_stock.timestamp)

In [18]:
final_data = final_stock.merge(final_covid, on = 'timestamp', how = 'left')

In [19]:
final_data.head()

,timestamp,open,high,low,close,volume,stock_name,Country/Region,death_cases,confirmed_cases
0,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Afghanistan,2449,55827
1,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Albania,1897,111301
2,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Algeria,3007,113948
3,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Andorra,112,10998
4,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Angola,511,21026


In [20]:
final_data.shape

(96000, 10)

In [21]:
final_data.timestamp.min()

Timestamp('2020-10-12 00:00:00')

In [22]:
final_data.timestamp.max()

Timestamp('2021-03-05 00:00:00')

In [23]:
final_data.isna().sum()

timestamp          0
open               0
high               0
low                0
close              0
volume             0
stock_name         0
Country/Region     0
death_cases        0
confirmed_cases    0
dtype: int64

In [24]:
MSFT_final = final_data[final_data.stock_name == 'MSFT']

In [25]:
MSFT_final.head()

,timestamp,open,high,low,close,volume,stock_name,Country/Region,death_cases,confirmed_cases
0,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Afghanistan,2449,55827
1,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Albania,1897,111301
2,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Algeria,3007,113948
3,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Andorra,112,10998
4,2021-03-05,229.5165,233.27,226.46,231.6,41872770,MSFT,Angola,511,21026
